In [1]:
import requests
import pandas as pd
from pathlib import Path
import datetime

OVERPASS_URL = "https://overpass-api.de/api/interpreter"

def build_query(place="Berlin", admin_boundary="administrative"):
    return f"""
[out:json][timeout:60];
area["name"="{place}"]["boundary"="{admin_boundary}"]->.searchArea;
(
  node["amenity"="veterinary"](area.searchArea);
  way["amenity"="veterinary"](area.searchArea);
  relation["amenity"="veterinary"](area.searchArea);
);
out center tags;
"""

def fetch_overpass(query, url=OVERPASS_URL):
    r = requests.post(url, data=query)
    r.raise_for_status()
    return r.json()

def elements_to_df(data):
    rows = []
    for el in data.get("elements", []):
        tags = el.get("tags", {}) or {}
        lat = el.get("lat") or (el.get("center") or {}).get("lat")
        lon = el.get("lon") or (el.get("center") or {}).get("lon")
        rows.append({
            "osmid": el.get("id"),
            "name": tags.get("name", ""),
            "amenity": tags.get("amenity", ""),
            "addr:street": tags.get("addr:street", ""),
            "addr:housenumber": tags.get("addr:housenumber", ""),
            "addr:postcode": tags.get("addr:postcode", ""),
            "addr:city": tags.get("addr:city", ""),
            "phone": tags.get("contact:phone", "") or tags.get("phone", ""),
            "website": tags.get("contact:website", "") or tags.get("website", ""),
            "email": tags.get("contact:email", "") or tags.get("email", ""),
            "opening_hours": tags.get("opening_hours", ""),
            "operator": tags.get("operator", "") or tags.get("brand", ""),
            "brand": tags.get("brand", ""),
            "veterinary:speciality": tags.get("veterinary:speciality", ""),
            "wheelchair": tags.get("wheelchair", ""),
            "emergency": tags.get("emergency", ""),
            "lat": lat,
            "lon": lon,
        })
    df = pd.DataFrame(rows)
    return df

def main(place="Berlin", out_dir="./vet_clinics/sources"):
    # 1) query Overpass
    q = build_query(place=place)
    data = fetch_overpass(q)

    # 2) normalize to DataFrame
    df = elements_to_df(data)

    # 3) keep only amenity=veterinary (safety) and drop exact osmid dups
    if not df.empty and "amenity" in df.columns:
        df = df[df["amenity"].str.lower() == "veterinary"].copy()
    if not df.empty and "osmid" in df.columns:
        df = df.drop_duplicates(subset=["osmid"], keep="first").copy()

    # 4) ensure full column set (add missing as empty), reorder
    cols = [
        "osmid","name","amenity",
        "addr:street","addr:housenumber","addr:postcode","addr:city",
        "phone","website","email",
        "opening_hours","operator","brand",
        "veterinary:speciality","wheelchair","emergency",
        "lat","lon",
    ]
    for c in cols:
        if c not in df.columns:
            df[c] = ""
    df = df[cols].copy()

    # 5) save (dated + stable)
    out_path = Path(out_dir)
    out_path.mkdir(parents=True, exist_ok=True)

    dated_csv = out_path / f"vets_osm_berlin_{datetime.date.today().isoformat()}.csv"
    stable_csv = out_path / "berlin_vet_clinics_osm.csv"

    df.to_csv(dated_csv, index=False, encoding="utf-8")
    df.to_csv(stable_csv, index=False, encoding="utf-8")

    print(f"Exported {len(df)} rows to:")
    print(f"  - {dated_csv.resolve()}")
    print(f"  - {stable_csv.resolve()}")

    return df

# ---- run it (notebook-friendly) ----
df = main()
df.head()


Exported 178 rows to:
  - /Users/martinsvitek/layered-populate-data-pool-da/layered-populate-data-pool-da/vet_clinics/sources/vet_clinics/sources/vets_osm_berlin_2025-09-25.csv
  - /Users/martinsvitek/layered-populate-data-pool-da/layered-populate-data-pool-da/vet_clinics/sources/vet_clinics/sources/berlin_vet_clinics_osm.csv


,osmid,name,amenity,addr:street,addr:housenumber,addr:postcode,addr:city,phone,website,email,opening_hours,operator,brand,veterinary:speciality,wheelchair,emergency,lat,lon
0,268917040,Tierarztpraxis am Urban,veterinary,Baerwaldstraße,69,10961,Berlin,,,,"Mo-Sa 10:00-12:00, Mo 17:00-19:00, Tu,We,Fr 16...",,,,no,,52.495684,13.405233
1,299795048,Dr. med. vet. Elke Hartwig,veterinary,Straße 48,67,13125,Berlin,+49 30 9437820,http://www.tierarztpraxis-hartwig.de/,,"Mo,Tu,Th,Fr 10:00-12:00, Mo-Fr 15:00-18:00",,,,limited,,52.606286,13.479555
2,347294456,Tierarztpraxis Dr. Bernhard Sörensen,veterinary,Königsberger Straße,36,12207,Berlin,+49 30 7738321,https://www.tierarztpraxis-soerensen.de/,,"Mo-Fr 09:00-20:00; Sa, Su 10:00-18:00",,,,yes,,52.429722,13.320133
3,394867279,Tierarztpraxis Jeanette Koepsel,veterinary,,,,,,,,,,,,,,52.535199,13.270573
4,411550894,Kleintierarztpraxis Berlin Kaulsdorf,veterinary,Planitzstraße,19,12621,Berlin,+49 30 53018585,https://www.tierarzt-kaulsdorf.de/,info@tierarzt-kaulsdorf.de,"Mo-Fr 09:00-19:00 open ""tel. Terminvereinbarun...",Dr. Berit Miels;Dr. Mathias Kochert,,,,,52.509511,13.589635
